<a href="https://colab.research.google.com/github/MarieThe5454/ReposName/blob/main/F2V3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Block 1: Import necessary libraries and load dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import shap

In [ ]:
# Automatically load and save dataset.
!wget "https://cloudstore.uni-ulm.de/s/R28JzBCTPEAdyRH/download" -O churn_data.csv

# Load the dataset as pandas dataframe
churn_data = pd.read_csv('churn_data.csv')

# Show dataset
churn_data

In [ ]:
# Block 2: Data overview and initial visualization
print("Dataset Information:")
print(churn_data.info())
print("\nSample Rows:")
print(churn_data.head())

# Visualize churn distribution
plt.figure(figsize=(6, 4))
churn_data['Churn'].value_counts(normalize=True).plot(kind='bar', color=['blue', 'orange'])
plt.title("Churn Distribution")
plt.xlabel("Churn (0 = No, 1 = Yes)")
plt.ylabel("Proportion")
plt.show()


In [ ]:
# Block 3: Correlation analysis and encoding categorical variables
# Convert categorical features into numerical using One-Hot-Encoding
churn_data_enc = pd.get_dummies(churn_data, columns=['Gender', 'Married'], drop_first=True)

# Visualize correlations between features
correlation_matrix = churn_data_enc.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

In [ ]:
# Block 4: Feature selection and target assignment
X = churn_data_enc.drop('Churn', axis=1)
y = churn_data_enc['Churn']

In [ ]:
# Block 5: Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
param_grid = {
    'n_estimators': [50, 100],  # Reduced range
    'max_depth': [10, 20],  # Reduced range
    'min_samples_split': [2, 5],  # Fewer values
    'min_samples_leaf': [1, 2]  # Fewer values
}

random_search = RandomizedSearchCV(
    rf_model, param_grid, n_iter=5, cv=5, scoring='f1', n_jobs=-1, random_state=42
)

random_search = RandomizedSearchCV(
    rf_model, param_grid, n_iter=10, cv=3, scoring='f1', n_jobs=-1, random_state=42
)
random_search.fit(X_train, y_train)

# Retrieve the best model
best_model = random_search.best_estimator_


In [ ]:
# Block 6: Hyperparameter tuning using RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier # Import the RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42) # Initialize the RandomForestClassifier model

param_grid = {
    'n_estimators': [50, 100],  # Reduced range
    'max_depth': [10, 20],  # Reduced range
    'min_samples_split': [2, 5],  # Fewer values
    'min_samples_leaf': [1, 2]  # Fewer values
}

random_search = RandomizedSearchCV(
    rf_model, param_grid, n_iter=5, cv=5, scoring='f1', n_jobs=-1, random_state=42
)

# You can remove the redundant RandomizedSearchCV instantiation below
# random_search = RandomizedSearchCV(
#     rf_model, param_grid, n_iter=10, cv=3, scoring='f1', n_jobs=-1, random_state=42
# )

random_search.fit(X_train, y_train)

# Retrieve the best model
best_model = random_search.best_estimator_

In [ ]:
# Block 7: Evaluate the model
# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Print classification report and confusion matrix
print("Classification Report:")
print(classification_report(y_test, y_pred))

conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Block 8: Visualize feature importance
feature_importances = pd.DataFrame(best_model.feature_importances_, index=X.columns, columns=['Importance'])
feature_importances.sort_values(by='Importance', ascending=False).plot(kind='bar', figsize=(10, 6))
plt.title("Feature Importance")
plt.show()

In [ ]:
# Block 9: SHAP analysis for model interpretability
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_test)

# Visualize SHAP summary plot
shap.summary_plot(shap_values[1], X_test, plot_type="bar")

In [ ]:
# Block 10: Recommendations based on insights
print("Recommendations:")
print("- Focus retention strategies on customers showing high risk of churn based on key features.")
print("- Enhance customer engagement for at-risk groups identified in feature analysis.")
print("- Provide targeted offers or support to improve customer satisfaction and reduce churn.")
